In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd

In [ ]:
data=pd.read_csv('../input/train_LZdllcl.csv')

In [ ]:
data.info()

In [ ]:
#Education and Previous year rating has missing values
#Try to find out the type of missing values in previous year rating type
data.previous_year_rating.value_counts()

In [ ]:
#Try to find out the relation between prev year rating and length_of_service
#No of missing value in prev year rating
def impute_missing_val(data):
    print('No of missing value in prev year rating=',len(data[data.previous_year_rating.isnull()].
                                                     previous_year_rating))
    data_null_re=data[['previous_year_rating','length_of_service']]
    print(data_null_re[data_null_re.previous_year_rating.isnull()].head(5))
    data_null_re[data_null_re.previous_year_rating.isnull()].groupby(['length_of_service']).count()
#CREATE A COLUMN AFTER RPLACING THE MISSING VALUE
    data_null_re.fillna(0,inplace=True)
    print(data_null_re.head(11))
#Count the missinng values in education column
    print("No. of missing values in education column",data.education.isnull().sum())
#Check the proportion of differnt education type
    print(data.education.value_counts())
    data_edu_null_re=data[['education']]
    data_edu_null_re.fillna("""Bachelor's""",inplace=True)
    data['education']=data_edu_null_re
    data['previous_year_rating']=data_null_re[['previous_year_rating']]
    #Drop the  employee_id column
    data.drop(['employee_id'],axis=1,inplace=True)
    return data

 As you can see all the missing values in prev_yr_rating column belongs to the person who has having length of service=1. It means his/her rating is not annnounced yet or he may not be elligible for rating. So we can replace the  missing values with 0 as rating in the prev_yr_rating column

Most of the people having Bachlero's degree in their hand. So we can replace the 2409 missing values with Bachelor degree. Although we can predict the missing values usinng ML algorithm but for simplicity we are simple replacinng with commonlyoccuring instance

In [ ]:
#Now you can see that none of the column having miissing value
data=impute_missing_val(data)
data.info()

In [ ]:
data.head()

You can see that most of the columns are categorical. For example Department,region,education,gender,
recruitment_channel,KPIs met,award won,previous year rating


In [ ]:
#Try to have some insights on the number of levels for each categorical feature
cat_features=['department','region','education','gender','recruitment_channel','previous_year_rating','KPIs_met >80%','awards_won?']
for column in data[cat_features]:
    #print(set(data[[column]]))
    print('Unique levels in {0} column'.format(str(column)),set(data[column]))

#It seems that region column has too many unique levels. Also some of the categorical features are  already coded in numerical 
form. So we need to covert department ,region ,education,gender,recruitment_channel into numerical form

In [ ]:
print('No. of unique levels in region column',len(list(set(data['region']))))

Please find the link https://towardsdatascience.com/smarter-ways-to-encode-categorical-data-for-machine-learning-part-1-of-3-6dca2f71b159.
https://towardsdatascience.com/understanding-feature-engineering-part-2-categorical-data-f54324193e63
From this link I can find some interesting methods to encode the categorical data

We have 
* 9 levels in Deparments
* 34 levels in region
* 3 levels each in education and sourcing channel
* 2 levels in gender,KPI met and award won columns
* Previous_year_rating column is already coded and it is ordinal so we need to code it in the same way

From OneHotEncoding there will be 9 features for dept,34 columns for region, 3-3 columns  for education and sourcing channel
so we have  49 additional feature

In [ ]:
#http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html
from sklearn.preprocessing import OneHotEncoder

In [ ]:
data_onehot_enc=data

**ONE HOT ENCODING**

In [ ]:

onehot_encoder=OneHotEncoder()
colms=['department','region','education','recruitment_channel']
onehot_encoded=onehot_encoder.fit_transform(data_onehot_enc[colms])
data_onehot_enc.drop(colms,inplace=True,axis=1)
#Change the encoded feature into df


In [ ]:
df_onehot_enc=pd.DataFrame(data=onehot_encoded.toarray(),columns=onehot_encoder.get_feature_names())

In [ ]:
#Covert Gender column using Label Encoder
from sklearn.preprocessing import LabelEncoder
label_encoded=LabelEncoder().fit_transform(data_onehot_enc['gender'])
data_onehot_enc['gender']=label_encoded


In [ ]:
df_onehot_enc=pd.concat([data_onehot_enc,df_onehot_enc],axis=1).info()

You can see that we have total 58 features using One Hot Encoding for the 'department','region','education','recruitment_channel' and we have assigned binary values in the gender column without creating two seperate columns for male and female

**LabelEncoder**

In [ ]:
data=pd.read_csv('../input/train_LZdllcl.csv')
data_label_enc=impute_missing_val(data)

In [ ]:
le_enc=LabelEncoder()
colms=['gender','department','region','education','recruitment_channel']
data_label_enc['gender']=le_enc.fit_transform(data_label_enc['gender'])
data_label_enc['department']=le_enc.fit_transform(data_label_enc['department'])
print(le_enc.classes_)
data_label_enc['region']=le_enc.fit_transform(data_label_enc['region'])
data_label_enc['education']=le_enc.fit_transform(data_label_enc['education'])
data_label_enc['recruitment_channel']=le_enc.fit_transform(data_label_enc['recruitment_channel'])

In [ ]:
data_label_enc.head()

As you can see  that each and every category in the  categorical column has been assigned a numerical value starting from 0. In this case  number of column remain same as in original data frame. As we can find in various articles that label encoding suffer from a problem i.e. ML model will treat these  numerical value in some order.For ex. It will give them 0,1 and 2. The problem with this approach is that there is no relation between these three classes yet our Algo might consider them to be ordered (that is there is some relation between them) maybe 0<1<2

**Binary Encooding**
* Convert each category in numeric form usinng label encoder
* Genearte binary number for each decimal digit
* Now fill the each digit of binary number into seperate column
* For example usinng label encoder we got decimal no. 0,1,2.....9 for dept. We can covert these numbers into 4  bit binary  no format
for ex. 8=1000,7=0111,9=1001. Since it requires 4 bit hence 4 columns are needed to accomodate each bit. Hence department column can be coded in just 4 column while one hot encoding requires 9 columns
* Binary encoding works well with higher  coordinality 
* It is a hybrid of one-hot and hashing encoders
* Genearally we should use binary encoding when Categories>3
* You should try this  coding scheme for ordinal data with higher cardinaallity

In [ ]:
data=pd.read_csv('../input/train_LZdllcl.csv')
data_binary_enc=impute_missing_val(data)

In [ ]:
data_binary_enc.department.value_counts()

In [ ]:
data_label_enc.department.value_counts()

In [ ]:
import category_encoders as ce
lb=ce.BinaryEncoder()
binarized_dept=lb.fit_transform(data_binary_enc[['department']])
lb=ce.BinaryEncoder()
binarized_region=lb.fit_transform(data_binary_enc[['region']])
lb=ce.BinaryEncoder()
binarized_gender=lb.fit_transform(data_binary_enc[['gender']])
lb=ce.BinaryEncoder()
binarized_channel=lb.fit_transform(data_binary_enc[['recruitment_channel']])
lb=ce.BinaryEncoder()
binarized_edu=lb.fit_transform(data_binary_enc[['education']])


In [ ]:
binarized_dept.describe()

In [ ]:
#You can see that after binary encoding region_0 and department_0 column is redindant
binarized_region.drop(['region_0'],axis=1,inplace=True)
binarized_dept.drop(['department_0'],axis=1,inplace=True)
binarized_gender.drop(['gender_0'],axis=1,inplace=True)
binarized_edu.drop(['education_0'],axis=1,inplace=True)
binarized_channel.drop(['recruitment_channel_0'],axis=1,inplace=True)

In [ ]:
data_binary_enc.drop(['gender','department','region','education','recruitment_channel'],axis=1,inplace=True)

In [ ]:
data_binary_enc=pd.concat([binarized_region,binarized_dept,binarized_gender,binarized_edu,binarized_channel,data_binary_enc],axis=1)

In [ ]:
data_binary_enc.info()

You can see that we have achieved 23 columns and memory usage =9.6 MB. Earlier in onehot encoding we had 58 columns and 23 MB memory usage

In [ ]:
data=pd.read_csv('../input/train_LZdllcl.csv')
df_hash_enc_data=impute_missing_val(data)


In [ ]:
ce_hash=ce.HashingEncoder(cols=['department','region','education','recruitment_channel'],
                          drop_invariant=True,n_components=10,return_df=True,hash_method='md5')
df_hash=ce_hash.fit_transform(df_hash_enc_data[['department','region','education','recruitment_channel']])

https://medium.com/value-stream-design/introducing-one-of-the-best-hacks-in-machine-learning-the-hashing-trick-bf6a9c8af18f
https://booking.ai/dont-be-tricked-by-the-hashing-trick-192a6aae3087
https://booking.ai/crunching-big-data-with-4-machine-learning-libraries-284ae3167885

https://blog.myyellowroad.com/using-categorical-data-in-machine-learning-with-python-from-dummy-variables-to-deep-category-66041f734512

In [ ]:
df_hash.describe()

In [ ]:
df_hash_enc_data.drop(['department','region','education','recruitment_channel'],axis=1,inplace=True)

In [ ]:
le=LabelEncoder()
df_hash_enc_data['gender']=le.fit_transform(df_hash_enc_data['gender'])

In [ ]:
df_hash_enc_data=pd.concat([df_hash,df_hash_enc_data],axis=1)
df_hash_enc_data.info()

You can see that we have total 19 columns includinng 1 target column. You can adjuste the number of columns return after hashinng using the parameter n_components in the HashingEncoderFunction

https://stats.idre.ucla.edu/r/library/r-library-contrast-coding-systems-for-categorical-variables/

I will update this article later with  some other techniques like Helmert coding,sum coding,Backward difference coding  etc